In [ ]:
import numpy as np
import optuna
import matplotlib.pyplot as plt
from bcmix import *

In [ ]:
ACTION_RANGE = (-5.0, 5.0)
N_TRIALS = 50
DATA_LEN = 80
GAMMA = 0.9

In [ ]:
# true value
alpha, beta = 1.8, -2.4

# prior
canonical_0 = np.array([[0.0], [0.0]])
precision_0 = np.array([[1.0, 0.0], [0.0, 1.0]])

print(alpha, beta, myopic(canonical_0, precision_0))

### Rollout

In [ ]:
def objective(trial):
    a = trial.suggest_float('a', ACTION_RANGE[0], ACTION_RANGE[1])
    q = q_myopic_without_change(canonical_i, precision_i, a, alpha, beta)
    return q

In [ ]:
for i in range(100):
    # initialize
    simresult_i = np.full((DATA_LEN * N_TRIALS, 7), np.nan)
    canonical_i, precision_i = canonical_0, precision_0
    for j in range(DATA_LEN):
        # current state
        covm_i = np.linalg.inv(precision_i)
        mean_i = covm_i @ canonical_i
        js = j * N_TRIALS
        je = j * N_TRIALS + N_TRIALS
        simresult_i[js : je, 0] = mean_i[0][0]
        simresult_i[js : je, 1] = mean_i[1][0]
        simresult_i[js : je, 2] = covm_i[0][0]
        simresult_i[js : je, 3] = covm_i[0][1]
        simresult_i[js : je, 4] = covm_i[1][1]
        # select action
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=N_TRIALS)
        simresult_i[js : je, 5] = [_.params['a'] for _ in study.trials]
        simresult_i[js : je, 6] = [_.value for _ in study.trials]
        a = study.best_trial.params['a']
        # update state
        y = env_response(a, alpha, beta)[0]
        canonical_i, precision_i = update_without_change(canonical_i, precision_i, a, y)
    np.save("simulations\sim_rollnoch_" + str(i) + ".npy", simresult_i)

### Plots

In [ ]:
qvalues = np.full((100, DATA_LEN), np.nan)
regrets = np.full((100, DATA_LEN), np.nan)
for i in range(100):
    simresult_i = np.load("simulations\sim_rollnoch_" + str(i) + ".npy")
    for j in range(DATA_LEN):
        simresult_ij = simresult_i[(j * N_TRIALS) : (j * N_TRIALS + N_TRIALS)]
        best = np.argmax(simresult_ij[:, 6])
        qvalues[i, j] = simresult_ij[best, 6]
        regrets[i, j] = (GAMMA ** j) * (alpha + simresult_ij[best, 5] * beta) ** 2

In [ ]:
# plot Q values
plt.plot(qvalues.mean(axis=0))

In [ ]:
# plot regret
plt.plot(np.cumsum(regrets.mean(axis=0)))